In [1]:
import optuna
import os
import trako as TKO
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
DATASETNAME = 'ADHD_STFC_diff_for_Daniel'
DATADIR = '/home/d/DATA/Dropbox/TRAKODATA/'+DATASETNAME+'/'
SUBJECTS = ['101','201']
GROUP = ['tracts_left_hemisphere']

In [3]:
import sys
sys.path.append('../trako')
import tuner

In [4]:
OUTPUTDIR = '/tmp/TRAKO_OPTUNA1/'
vtpsizes = []
tkosizes = []
for S in SUBJECTS[0:1]:
    for G in GROUP[0:1]:
        CURRENTDIR = os.path.join(DATADIR, S, G)
        CURRENTOUTDIR = os.path.join(OUTPUTDIR, S, G)
        files = os.listdir(CURRENTDIR)[0:1]
        
        if not os.path.exists(CURRENTOUTDIR):
            os.makedirs(CURRENTOUTDIR)
        
        for f in files:
            if not f.endswith('vtp'):
                continue
            
            vtpfile = os.path.join(CURRENTDIR, f)

            search_space = {
             'position': [True,False],
             'sequential': [True,False],
             'quantization_bits': list(range(15,25)),
             'compression_level': list(range(3,7))
            }
            
            tuner.Tuner.tune(vtpfile, search_space, maxdelta=0.01)

VTP size 1172155
TKO default size 377009


[I 2020-01-30 10:27:10,092] Setting status of trial#0 as TrialState.PRUNED. 
[I 2020-01-30 10:27:12,184] Setting status of trial#1 as TrialState.PRUNED. 
[I 2020-01-30 10:27:12,554] Setting status of trial#2 as TrialState.PRUNED. 
[I 2020-01-30 10:27:16,263] Finished trial#3 resulted in value: 786391.0. Current best value is 786391.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 21, 'compression_level': 6}.
[I 2020-01-30 10:27:16,503] Setting status of trial#4 as TrialState.PRUNED. 
[I 2020-01-30 10:27:20,964] Finished trial#5 resulted in value: 922591.0. Current best value is 786391.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 21, 'compression_level': 6}.
[I 2020-01-30 10:27:22,295] Setting status of trial#6 as TrialState.PRUNED. 
[I 2020-01-30 10:27:26,075] Finished trial#7 resulted in value: 740855.0. Current best value is 740855.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 20, 'com

[I 2020-01-30 10:29:02,032] Setting status of trial#82 as TrialState.PRUNED. 
[I 2020-01-30 10:29:03,483] Setting status of trial#83 as TrialState.PRUNED. 
[I 2020-01-30 10:29:04,030] Setting status of trial#84 as TrialState.PRUNED. 
[I 2020-01-30 10:29:04,421] Setting status of trial#85 as TrialState.PRUNED. 
[I 2020-01-30 10:29:05,897] Setting status of trial#86 as TrialState.PRUNED. 
[I 2020-01-30 10:29:06,287] Setting status of trial#87 as TrialState.PRUNED. 
[I 2020-01-30 10:29:07,706] Setting status of trial#88 as TrialState.PRUNED. 
[I 2020-01-30 10:29:09,132] Setting status of trial#89 as TrialState.PRUNED. 
[I 2020-01-30 10:29:09,568] Setting status of trial#90 as TrialState.PRUNED. 
[I 2020-01-30 10:29:13,461] Finished trial#91 resulted in value: 831843.0. Current best value is 649395.0 with parameters: {'position': False, 'sequential': True, 'quantization_bits': 18, 'compression_level': 6}.
[I 2020-01-30 10:29:13,720] Setting status of trial#92 as TrialState.PRUNED. 
[I 2020

ValueError: All grids have been evaluated. If you want to avoid this error, please make sure that unnecessary trials do not run during optimization by properly setting `n_trials` in `study.optimize`.

In [15]:
study.best_trial

FrozenTrial(number=6, state=TrialState.COMPLETE, value=111336.0, datetime_start=datetime.datetime(2020, 1, 30, 8, 50, 20, 536952), datetime_complete=datetime.datetime(2020, 1, 30, 8, 50, 20, 921601), params={'position': False, 'sequential': True, 'quantization_bits': 6, 'compression_level': 0}, distributions={'position': CategoricalDistribution(choices=(True, False)), 'sequential': CategoricalDistribution(choices=(True, False)), 'quantization_bits': IntUniformDistribution(high=31, low=0), 'compression_level': IntUniformDistribution(high=10, low=0)}, user_attrs={}, system_attrs={'_number': 6}, intermediate_values={}, trial_id=6)

In [16]:
study.best_trial.params

{'position': False,
 'sequential': True,
 'quantization_bits': 6,
 'compression_level': 0}

In [18]:
import imp, os
tkompare = imp.load_source('tkompare', '../tkompare')

config = {
  '*': { 
    'position':study.best_trial.params['position'],
    'sequential':study.best_trial.params['sequential'],
    'quantization_bits':study.best_trial.params['quantization_bits'],
    'compression_level':study.best_trial.params['compression_level'],
    'quantization_range':-1,
    'quantization_origin':None
  }

}



tkompare.compare(vtpfile, '/tmp/restored.vtp')

+-------------------+---------+-------------+
|        File       |  Bytes  | Streamlines |
+-------------------+---------+-------------+
| cluster_00616.vtp | 1172155 |     190     |
|    restored.vtp   |  974914 |     190     |
+-------------------+---------+-------------+
+--------------+------------+------------+-------------+----------+
|   Geometry   | Min. Delta | Max. Delta | Mean. Delta | StdDev.  |
+--------------+------------+------------+-------------+----------+
|  XYZ Points  |  36.01881  | 121.135544 |   66.63499  | 14.55739 |
| Line Indices |    0.0     |    0.0     |     0.0     |   0.0    |
+--------------+------------+------------+-------------+----------+
+---------------------------------+------------+------------+-------------+-----------+
|             Scalars             | Min. Delta | Max. Delta | Mean. Delta |  StdDev.  |
+---------------------------------+------------+------------+-------------+-----------+
| NormalizedSignalEstimationError |    0.0     |  0.